## LSTM-Inception Text Classification with Keras

In [ ]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [ ]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### LSTM + Inception Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), [CNN Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Deeper CNN as described in [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [ ]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

l_lstm1 = Bidirectional(LSTM(4,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))(embedded_sequences)

# inception module
inception, filter_sizes = [], [3,5,7]
for fsz in filter_sizes:
    l_conv_i1 = Conv1D(filters=12,kernel_size=1,
                    activation='relu',)(l_lstm1)
    l_conv_i2 = Conv1D(filters=12,kernel_size=fsz,
                    activation='relu',)(l_conv_i1)
    inception.append(l_conv_i2)
l_pool_inc = MaxPooling1D(3)(l_lstm1)
l_conv_inc = Conv1D(filters=12,kernel_size=1,
                    activation='relu',)(l_pool_inc)
inception.append(l_conv_inc)

l_merge = Concatenate(axis=1)(inception)
l_pool1 = MaxPooling1D(2)(l_merge)
l_drop1 = Dropout(0.3)(l_pool1)
l_flat = Flatten()(l_drop1)
l_dense = Dense(16, activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_flat)

preds = Dense(4, activation='softmax')(l_dense)

In [ ]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
#keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc', lr_metric])

In [ ]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [ ]:
model.summary()
model.save('ltsm-inception.h5')
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

In [ ]:
model.save('ltsm-inception.h5')